# Semana 7 do Aceleradev DS Codenation

### Professor: Kazuki Yokoyama | Tema: Feature Engineering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## O que é Feature Engeeniring?

Todo modelo de aprendizado de máquinas é alimentado com dados, que precisam ser trabalhados para que o modelo aprenda adequadamente e retorne boa predições. Isso é *engenharia de features*! muitas vezes os dados estão fora de escala, ou um dado categórico, se transformado em categoria numérica, pode enriquecer os dados de treino. Essa habilidade do DS é muito importante no processo, pois os dados não vêm *bonitinhos* prontos para serem consumidos, eles precisam ser *engenheirados* primeiro, e grande parte do tempo gasto na resolução de um problema com ML, é nessa etapa. Diante disso, a qualidade dela é diretamente proporcional a do modelo preditivo

**Fixando uma seed**

In [2]:
np.random.seed(1000)

**Criação de DataFrame Aleatório com variáveis categóricas e numéricas**

In [3]:
linhas = 100

In [4]:
altura = np.random.normal(loc=1.7, scale=.2, size=linhas).round(3)
ponto = np.random.normal(loc=7, scale=1,size=linhas).round(3)
cursos = ['mat', 'fis', 'bio']
curso = np.random.choice(a=cursos, size=linhas)

In [5]:
df = pd.DataFrame({'altura': altura,
                  'ponto': ponto,
                  'curso': curso})
df.head()

,altura,ponto,curso
0,1.539,6.610,bio
1,1.764,6.419,bio
2,1.695,8.949,bio
3,1.829,5.137,bio
4,1.640,6.432,fis


`df` é uma tabela que contém três variáveis, duas **numéricas** e uma **categórica**

## Começando a Engenharia de Features

### Variáveis Categóricas

**One-hot Encoding:** As $n$ categorias da variável categórica serão usadas para criação de novas colunas na sua tabela. Cada categoria representará uma coluna. Como uma tupla consegue ter apenas uma categoria, a coluna correspondente à categoria terá o valor 1 (Hot) preenchido, enquanto as demais terão 0 (Cold) preenchido. Perceba que teremos novas variáveis **binarias** em nosso `df`

Para fazer isso, usaremos uma classe do sklearn.preprocessing, o **OneHotEncoder**

```python
sklearn.preprocessing.OneHotEncoder(sparse=False, dtype=np.uint8)
```

**Nota:** caso deixe `sparse=True`, vc terá uma matriz `sparse matrix`. Ela economiza mais memória, mas não mostra a matriz diretamente. Para revelar então

```python
sparse_matrix.toarray()
```

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
one_hot_enc = OneHotEncoder(sparse=False, dtype=np.uint8)

In [8]:
course_enc = one_hot_enc.fit_transform(df[['curso']])  # lembre de colocar [[]] pro atributo entender que é um (n x 1)

In [9]:
course_enc.shape

(100, 3)

In [10]:
course_enc[:5]

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]], dtype=uint8)

In [11]:
one_hot_enc.categories_

[array(['bio', 'fis', 'mat'], dtype=object)]

In [12]:
cat_columns = one_hot_enc.categories_[0]

Podemos concatenar essas informações ao `df`

In [13]:
df_dois = df.join(pd.DataFrame(course_enc, columns=cat_columns))
df_dois.head()

,altura,ponto,curso,bio,fis,mat
0,1.539,6.610,bio,1,0,0
1,1.764,6.419,bio,1,0,0
2,1.695,8.949,bio,1,0,0
3,1.829,5.137,bio,1,0,0
4,1.640,6.432,fis,0,1,0


**Binarização:** Processo que discretiza uma variável numérica contínua, tendo um certo *limite* como parâmetro. O valor **acima do limite** recebe 1, já **abaixo do limite** recebe 0

Para fazer isso, usaremos uma classe do sklearn.preprocessing, o **Binarizer**
```python
sklearn.preprocessing.Binarizer(threshold=float)
```

In [14]:
from sklearn.preprocessing import Binarizer

O limite para `altura` será de 1.8

In [15]:
binarizer = Binarizer(threshold=1.8)

In [16]:
binary = binarizer.fit_transform(df[['altura']])

In [17]:
binary.flatten()

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1.])

In [18]:
df_tres = df_dois.join(pd.Series(binary.flatten(), name='alto'))
df_tres.head()

,altura,ponto,curso,bio,fis,mat,alto
0,1.539,6.610,bio,1,0,0,0.0
1,1.764,6.419,bio,1,0,0,0.0
2,1.695,8.949,bio,1,0,0,0.0
3,1.829,5.137,bio,1,0,0,1.0
4,1.640,6.432,fis,0,1,0,0.0


**Intervalo de Classes (Binnig):** Também conhecido como discretização, seu papel é categorizar variáveis numéricas contínuas, sendo a categoria um intervalo que contém a informação

$$2.456 \subset 2\vdash3$$

Com **sklearn**:
```python
sklearn.preprocessing.KBinsDiscretizer(n_bins=int, encode=, strategy=)
```
*Parâmetros:*

- encode: {'onehot', 'ordinal'}
    - 'onehot': resulta uma `sparse matrix` de *one-hot encoding*
    - 'ordinal': resulta inteiros que correspondem à qual intervalo de classe o valor pertence
- strategy: {'uniform', 'quantile'}
    - uniform: faz intervalos de classe de tamanhos iguais
    - quantile: faz intervalos de modo a terem a mesma frequência

**Atributos:**

- KBinsDiscretizer().bin_edges_: retorna um array com os limites dos intervalos de classes

Com **pandas**
```python
pd.cut(x=1-d array, bins=, labels=None)
```
*Parâmetros:*

- bins: int ou list
    - int: gera *int* intervalos de classes
    - list: usa os valores de *list* para fazer os intervalos
- labels: list ou None
    - list: podem ser uma lista de *strings*. Para bins=3, labels=['ruim', 'medio', 'bom']

In [112]:
from sklearn.preprocessing import KBinsDiscretizer

In [113]:
binning = KBinsDiscretizer(n_bins=4, encode='onehot', strategy='uniform')

In [114]:
one_hot_binning = binning.fit_transform(df[['ponto']]).toarray()  # colunas onehot

In [115]:
limites_intervalos = binning.bin_edges_
limites_intervalos[0]

array([4.094  , 5.39025, 6.6865 , 7.98275, 9.279  ])

In [141]:
intervalos = pd.cut(df_tres['ponto'], bins=4)
intervalos.unique()

[(5.39, 6.687], (7.983, 9.279], (4.089, 5.39], (6.687, 7.983]]
Categories (4, interval[float64]): [(4.089, 5.39] < (5.39, 6.687] < (6.687, 7.983] < (7.983, 9.279]]

**Nota:** use os intervalos do **cut**, mesmo sendo possível passar os advindos do **KBinsDiscretizer()**. Devido à arredondamentos diferentes, pode ser que apareçam valores com NaN. Portanto, na função **pd.cut()**, use a **mesma quantidade de bins** passada em **KBinsDiscretizer()**

In [146]:
df_onehot = pd.DataFrame(one_hot_binning, columns=intervalos.unique())
df_quatro = df_tres.join(df_onehot)
df_quatro.head()

,altura,ponto,curso,bio,fis,mat,alto,"(5.39, 6.687]","(7.983, 9.279]","(4.089, 5.39]","(6.687, 7.983]"
0,1.539,6.610,bio,1,0,0,0.0,0.0,1.0,0.0,0.0
1,1.764,6.419,bio,1,0,0,0.0,0.0,1.0,0.0,0.0
2,1.695,8.949,bio,1,0,0,0.0,0.0,0.0,0.0,1.0
3,1.829,5.137,bio,1,0,0,1.0,1.0,0.0,0.0,0.0
4,1.640,6.432,fis,0,1,0,0.0,0.0,1.0,0.0,0.0


Seria interessante usar o atributo *labels* de pd.cut(). Como são notas, poderiam ser ['ruim', 'medio', 'bom', 'excelente']

Embora tenhamos lidado tanto com variáveis já categóricas quanto numéricas, a idéia era torná-las **categóricas** de uma maneira que um modelo entenda. E fizemos com o **one-hot encoding** e **binarização**, dando a possibilidade de **binarizar** esses tipos de variáveis

### Variáveis Numéricas

Agora, olharemos para as variáveis numéricas com o intuito de deixá-las com a mesma ordem de grandeza, pois alguns modelos são sensíveis à dados que possuem ordens de diferença muito grandes